## Preprocessing the Data

In [1]:
import numpy as np
from sklearn import preprocessing
import os

In [35]:
path = os.getcwd()+'/Data/'
raw_csv_data = np.loadtxt(os.getcwd()+'/Data/Audiobooks_data.csv',delimiter=',')
unscaled_inputs_all = raw_csv_data[:,1:-1]
targets_all = raw_csv_data[:,-1]

In [36]:
# shuffle the index
shuffled_indices = np.arange(unscaled_inputs_all.shape[0])
np.random.shuffle(shuffled_indices)

# Use the shuffled indices to shuffle the inputs and targets.
unscaled_inputs_all = unscaled_inputs_all[shuffled_indices]
targets_all = targets_all[shuffled_indices]

In [37]:
# balance the dataset
num_one_targets = int(np.sum(targets_all))
zero_targets_counter = 0
indices_to_remove = []
for i in range(targets_all.shape[0]):
    if targets_all[i] == 0:
        zero_targets_counter += 1
        if zero_targets_counter > num_one_targets:
            indices_to_remove.append(i)
unscaled_inputs_equal_priors = np.delete(unscaled_inputs_all, indices_to_remove, axis=0)
targets_equal_priors = np.delete(targets_all, indices_to_remove, axis=0)

In [38]:
# Standardize the inputs
scaled_inputs = preprocessing.scale(unscaled_inputs_equal_priors)

In [39]:
# shuffle the data
shuffled_indices = np.arange(scaled_inputs.shape[0])
np.random.shuffle(shuffled_indices)

# Use the shuffled indices to shuffle the inputs and targets.
shuffled_inputs = scaled_inputs[shuffled_indices]
shuffled_targets = targets_equal_priors[shuffled_indices]

In [40]:
# Split the dataset into train, validation, and test
samples_count = shuffled_inputs.shape[0]

train_samples_count = int(0.8 * samples_count)
validation_samples_count = int(0.1 * samples_count)

test_samples_count = samples_count - train_samples_count - validation_samples_count

train_inputs = shuffled_inputs[:train_samples_count]
train_targets = shuffled_targets[:train_samples_count]

validation_inputs = shuffled_inputs[train_samples_count:train_samples_count+validation_samples_count]
validation_targets = shuffled_targets[train_samples_count:train_samples_count+validation_samples_count]

test_inputs = shuffled_inputs[train_samples_count+validation_samples_count:]
test_targets = shuffled_targets[train_samples_count+validation_samples_count:]

print(np.sum(train_targets), train_samples_count, np.sum(train_targets) / train_samples_count)
print(np.sum(validation_targets), validation_samples_count, np.sum(validation_targets) / validation_samples_count)
print(np.sum(test_targets), test_samples_count, np.sum(test_targets) / test_samples_count)

1787.0 3579 0.4993014808605756
241.0 447 0.5391498881431768
209.0 448 0.46651785714285715


In [42]:
# save in .npz 
np.savez(path+'Audiobooks_data_train', inputs=train_inputs, targets=train_targets)
np.savez(path+'Audiobooks_data_validation', inputs=validation_inputs, targets=validation_targets)
np.savez(path+'Audiobooks_data_test', inputs=test_inputs, targets=test_targets)

## Load the Data

In [43]:
import numpy as np
import tensorflow as tf

In [46]:
npz = np.load(path+'Audiobooks_data_train.npz')
train_inputs = npz['inputs'].astype(np.float64)
train_targets = npz['targets'].astype(np.int32)

npz = np.load(path+'Audiobooks_data_validation.npz')
validation_inputs, validation_targets = npz['inputs'].astype(np.float64), npz['targets'].astype(np.int32)

npz = np.load(path+'Audiobooks_data_test.npz')
test_inputs, test_targets = npz['inputs'].astype(np.float64), npz['targets'].astype(np.int32)

## Model
Outline, optimizers, loss, early stopping and training

In [47]:
input_size = 10
output_size = 2
hidden_layer_size = 50

model = tf.keras.Sequential([
    # tf.keras.layers.Dense is basically implementing: output = activation(dot(input, weight) + bias)
    # it takes several arguments, but the most important ones for us are the hidden_layer_size and the activation function
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), # 1st hidden layer
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), # 2nd hidden layer
    # the final layer is no different, we just make sure to activate it with softmax
    tf.keras.layers.Dense(output_size, activation='softmax') # output layer
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [49]:
# Training
batch_size = 100
max_epochs = 100
early_stopping = tf.keras.callbacks.EarlyStopping(patience=2)

model.fit(train_inputs, 
          train_targets, 
          batch_size=batch_size, 
          epochs=max_epochs, 
          callbacks=[early_stopping],
          validation_data=(validation_inputs, validation_targets), 
          verbose = 2 
          )  

Epoch 1/100
36/36 - 0s - 5ms/step - accuracy: 0.8139 - loss: 0.3439 - val_accuracy: 0.8300 - val_loss: 0.3354
Epoch 2/100
36/36 - 0s - 2ms/step - accuracy: 0.8170 - loss: 0.3409 - val_accuracy: 0.8367 - val_loss: 0.3087
Epoch 3/100
36/36 - 0s - 2ms/step - accuracy: 0.8164 - loss: 0.3360 - val_accuracy: 0.8546 - val_loss: 0.2958
Epoch 4/100
36/36 - 0s - 2ms/step - accuracy: 0.8240 - loss: 0.3381 - val_accuracy: 0.8412 - val_loss: 0.3062
Epoch 5/100
36/36 - 0s - 2ms/step - accuracy: 0.8181 - loss: 0.3377 - val_accuracy: 0.8479 - val_loss: 0.3022


## Test the Model

In [50]:
test_loss, test_accuracy = model.evaluate(test_inputs, test_targets)

14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8333 - loss: 0.3323 


In [ ]:
print('\nTest loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))